In [8]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
import random

Data = []

In [17]:
start_time = time.time()
Images_Folder = "/Users/abhinavmathur/Downloads/kagglecatsanddogs_3367a/PetImages"

Animals = ["Cat","Dog"] #Specifying the 2 image classes that exist in the folder


for animal in Animals:
    classtype = Animals.index(animal) #Assigning a label encoding to the category
    path = Images_Folder+"/"+animal #Combining paths
    for img in os.listdir(path):
        try:
            animal_array = cv2.imread(path+"/"+img, cv2.IMREAD_GRAYSCALE) #Reading image as a grayscale image only
            #plt.imshow(animal_array, cmap="gray")
            new_array = cv2.resize(animal_array,(50,50)) #Resizing images to make them of a similar size
            Data.append([new_array,classtype]) #Creating a training array
        except Exception as e: #Simply skip over the exceptions, added this later
            pass
        
print("--- %s seconds taken to load data---" % (time.time() - start_time))

--- 65.76765203475952 seconds taken to load data---


In [21]:
# Need to reshuffle the data as the first N observations would belong to a single classification

random.shuffle(Data)

In [22]:
# Checking if the sampling worked correctly

for sample in Data[:10]:
    print(sample[1])

1
1
1
1
0
0
1
0
1
1


In [23]:
#Reshaping the data for classification

X = []
Y = []

for features,label in Data:
    X.append(features)
    Y.append(label)
    

In [28]:
X

[array([[ 15,   6,   3, ...,  11,  12,   3],
        [ 34,  15,   5, ...,  31,  11,   2],
        [ 46,  34,   2, ..., 119,  63,   3],
        ...,
        [  7,  20,   8, ...,  30,  27,  33],
        [ 18,  20,  13, ...,  30,  29,  41],
        [ 25,  25,  22, ...,  32,  69,  29]], dtype=uint8),
 array([[154, 148, 150, ..., 202, 144, 168],
        [146, 138, 155, ..., 146, 139, 138],
        [150, 148, 147, ..., 152, 131, 124],
        ...,
        [105, 152, 115, ..., 104, 110, 113],
        [109, 117, 112, ..., 120, 112,  98],
        [112, 102, 125, ..., 112, 118, 116]], dtype=uint8),
 array([[ 98,  90,  85, ...,  62,  72,  74],
        [ 93,  90,  86, ...,  66,  73,  73],
        [ 91,  88,  87, ...,  74,  73,  74],
        ...,
        [255, 255, 255, ..., 241, 244, 239],
        [253, 255, 254, ..., 239, 240, 237],
        [221, 235, 254, ..., 236, 231, 230]], dtype=uint8),
 array([[ 38,  40,  42, ...,  62,  62,  59],
        [ 38,  40,  42, ...,  78,  71,  59],
        [ 39,  3

In [29]:
X = np.array(X).reshape(-1,50,50,1)

In [30]:
X

array([[[[ 15],
         [  6],
         [  3],
         ...,
         [ 11],
         [ 12],
         [  3]],

        [[ 34],
         [ 15],
         [  5],
         ...,
         [ 31],
         [ 11],
         [  2]],

        [[ 46],
         [ 34],
         [  2],
         ...,
         [119],
         [ 63],
         [  3]],

        ...,

        [[  7],
         [ 20],
         [  8],
         ...,
         [ 30],
         [ 27],
         [ 33]],

        [[ 18],
         [ 20],
         [ 13],
         ...,
         [ 30],
         [ 29],
         [ 41]],

        [[ 25],
         [ 25],
         [ 22],
         ...,
         [ 32],
         [ 69],
         [ 29]]],


       [[[154],
         [148],
         [150],
         ...,
         [202],
         [144],
         [168]],

        [[146],
         [138],
         [155],
         ...,
         [146],
         [139],
         [138]],

        [[150],
         [148],
         [147],
         ...,
         [152],
         [

In [32]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [34]:
model = Sequential()
model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, Y, batch_size=32, epochs=3, validation_split=0.3)

Train on 17462 samples, validate on 7484 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/3
17462/17462 [==============================] - 761s 44ms/sample - loss: 7.9075 - acc: 0.5034 - val_loss: 8.1097 - val_acc: 0.4913
Epoch 2/3
17462/17462 [==============================] - 745s 43ms/sample - loss: 7.9146 - acc: 0.5036 - val_loss: 8.1097 - val_acc: 0.4913
Epoch 3/3
17462/17462 [==============================] - 719s 41ms/sample - loss: 7.9146 - acc: 0.5036 - val_loss: 8.1097 - val_acc: 0.4913


In [37]:
#Evaluation of model metrics

from tensorflow.keras.callbacks import TensorBoard

Name = "Cat-Dog-classifier-CNN"

tensorboard = TensorBoard(log_dir = "logs/{}".format(Name))

model.fit(X, Y,
          batch_size=32,
          epochs=3,
          validation_split=0.3,
          callbacks=[tensorboard])

Train on 17462 samples, validate on 7484 samples
Epoch 1/3
17462/17462 [==============================] - 2715s 155ms/sample - loss: 7.9146 - acc: 0.5036 - val_loss: 8.1097 - val_acc: 0.4913
Epoch 2/3
17462/17462 [==============================] - 990s 57ms/sample - loss: 7.9146 - acc: 0.5036 - val_loss: 8.1097 - val_acc: 0.4913
Epoch 3/3
17462/17462 [==============================] - 2908s 167ms/sample - loss: 7.9146 - acc: 0.5036 - val_loss: 8.1097 - val_acc: 0.4913
